## Installation

In [1]:
!nvidia-smi

Tue Sep 17 04:11:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!kaggle datasets download -d robikscube/flight-delay-dataset-20182022

Dataset URL: https://www.kaggle.com/datasets/robikscube/flight-delay-dataset-20182022
License(s): CC0-1.0
100% 3.73G/3.73G [00:45<00:00, 112MB/s]
100% 3.73G/3.73G [00:45<00:00, 88.6MB/s]


In [3]:
!unzip flight-delay-dataset-20182022.zip

Archive:  flight-delay-dataset-20182022.zip
  inflating: Airlines.csv            
  inflating: Combined_Flights_2018.csv  
  inflating: Combined_Flights_2018.parquet  
  inflating: Combined_Flights_2019.csv  
  inflating: Combined_Flights_2019.parquet  
  inflating: Combined_Flights_2020.csv  
  inflating: Combined_Flights_2020.parquet  
  inflating: Combined_Flights_2021.csv  
  inflating: Combined_Flights_2021.parquet  
  inflating: Combined_Flights_2022.csv  
  inflating: Combined_Flights_2022.parquet  
  inflating: raw/Flights_2018_1.csv  
  inflating: raw/Flights_2018_10.csv  
  inflating: raw/Flights_2018_11.csv  
  inflating: raw/Flights_2018_12.csv  
  inflating: raw/Flights_2018_2.csv  
  inflating: raw/Flights_2018_3.csv  
  inflating: raw/Flights_2018_4.csv  
  inflating: raw/Flights_2018_5.csv  
  inflating: raw/Flights_2018_6.csv  
  inflating: raw/Flights_2018_7.csv  
  inflating: raw/Flights_2018_8.csv  
  inflating: raw/Flights_2018_9.csv  

In [6]:
!pip install polars==1.5
!pip install "cudf_polars_cu12-24.8.0a433-py3-none-any.whl" --extra-index-url=https://pypi.anaconda.org/rapidsai-wheels-nightly/simple

Looking in indexes: https://pypi.org/simple, https://pypi.anaconda.org/rapidsai-wheels-nightly/simple
Processing ./cudf_polars_cu12-24.8.0a433-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.8/517.8 MB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.0 MB/s eta 0:00:00
  Created wheel for rmm-cu12: filename=rmm_cu12-24.8.2-cp310-cp310-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl size=1743566 sha256=6c6dc399dd24e4cc418280f1faca1d74b1a852e25e7798bf6bccf5e1d64f0c01
  Stored in directory: /root/.cache/pip/wheels/52/d3/4a/625ea3bd859c1353a9ca69712a3d58f8e83d8ac26ca7311983
Successfully built rmm-cu12
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: rmm-cu12
    Found existin

In [5]:
!pip install -q ipython-autotime
%load_ext autotime

time: 369 µs (started: 2024-09-17 04:16:02 +00:00)


## Imports

In [30]:
import polars as pl
from polars.testing import assert_frame_equal

time: 462 µs (started: 2024-09-17 04:47:34 +00:00)


In [31]:
pl.Config.set_fmt_float("full")

polars.config.Config

time: 30.6 ms (started: 2024-09-17 04:47:35 +00:00)


In [32]:
pl.__version__

'1.5.0'

time: 3.26 ms (started: 2024-09-17 04:47:35 +00:00)


## Reading the Dataset

In [33]:
import os

def get_csv_size_in_gb(filepath):
    """Gets the size of a CSV file in GB."""

    size_in_bytes = os.path.getsize(filepath)
    size_in_gb = size_in_bytes / (1024 ** 3)  # 1GB = 1024^3 bytes
    return size_in_gb

filepath = "Combined_Flights_2018.csv"  # Replace with the actual path to your CSV file
size_in_gb = get_csv_size_in_gb(filepath)

print(f"The size of {filepath} is: {size_in_gb:.2f} GB")

The size of Combined_Flights_2018.csv is: 1.86 GB
time: 1.63 ms (started: 2024-09-17 04:47:40 +00:00)


In [34]:
flight_df = pl.scan_csv("Combined_Flights_2018.csv")

time: 9.83 ms (started: 2024-09-17 04:47:41 +00:00)


In [35]:
flight_df.collect_schema()

Schema([('FlightDate', String),
        ('Airline', String),
        ('Origin', String),
        ('Dest', String),
        ('Cancelled', Boolean),
        ('Diverted', Boolean),
        ('CRSDepTime', Int64),
        ('DepTime', Float64),
        ('DepDelayMinutes', Float64),
        ('DepDelay', Float64),
        ('ArrTime', Float64),
        ('ArrDelayMinutes', Float64),
        ('AirTime', Float64),
        ('CRSElapsedTime', Float64),
        ('ActualElapsedTime', Float64),
        ('Distance', Float64),
        ('Year', Int64),
        ('Quarter', Int64),
        ('Month', Int64),
        ('DayofMonth', Int64),
        ('DayOfWeek', Int64),
        ('Marketing_Airline_Network', String),
        ('Operated_or_Branded_Code_Share_Partners', String),
        ('DOT_ID_Marketing_Airline', Int64),
        ('IATA_Code_Marketing_Airline', String),
        ('Flight_Number_Marketing_Airline', Int64),
        ('Operating_Airline', String),
        ('DOT_ID_Operating_Airline', Int64),
        

time: 123 ms (started: 2024-09-17 04:47:41 +00:00)


In [36]:
flight_df.head(5).collect()

FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,Operating_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,DestCityName,DestState,DestStateFips,DestStateName,DestWac,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
str,str,str,str,bool,bool,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,str,str,i64,str,i64,str,i64,str,str,i64,i64,i64,i64,str,str,i64,str,i64,i64,i64,i64,str,str,i64,str,i64,f64,f64,str,f64,f64,f64,f64,i64,f64,f64,f64,str,i64,f64
"""2018-01-23""","""Endeavor Air Inc.""","""ABY""","""ATL""",false,false,1202,1157,0,-5,1256,0,38,62,59,145,2018,1,1,23,2,"""DL""","""DL_CODESHARE""",19790,"""DL""",3298,"""9E""",20363,"""9E""","""N8928A""",3298,10146,1014602,30146,"""Albany, GA""","""GA""",13,"""Georgia""",34,10397,1039707,30397,"""Atlanta, GA""","""GA""",13,"""Georgia""",34,0,-1,"""1200-1259""",14,1211,1249,7,1304,-8,0,-1,"""1300-1359""",1,0
"""2018-01-24""","""Endeavor Air Inc.""","""ABY""","""ATL""",false,false,1202,1157,0,-5,1258,0,36,62,61,145,2018,1,1,24,3,"""DL""","""DL_CODESHARE""",19790,"""DL""",3298,"""9E""",20363,"""9E""","""N800AY""",3298,10146,1014602,30146,"""Albany, GA""","""GA""",13,"""Georgia""",34,10397,1039707,30397,"""Atlanta, GA""","""GA""",13,"""Georgia""",34,0,-1,"""1200-1259""",13,1210,1246,12,1304,-6,0,-1,"""1300-1359""",1,0
"""2018-01-25""","""Endeavor Air Inc.""","""ABY""","""ATL""",false,false,1202,1153,0,-9,1302,0,40,62,69,145,2018,1,1,25,4,"""DL""","""DL_CODESHARE""",19790,"""DL""",3298,"""9E""",20363,"""9E""","""N8836A""",3298,10146,1014602,30146,"""Albany, GA""","""GA""",13,"""Georgia""",34,10397,1039707,30397,"""Atlanta, GA""","""GA""",13,"""Georgia""",34,0,-1,"""1200-1259""",18,1211,1251,11,1304,-2,0,-1,"""1300-1359""",1,0
"""2018-01-26""","""Endeavor Air Inc.""","""ABY""","""ATL""",false,false,1202,1150,0,-12,1253,0,35,62,63,145,2018,1,1,26,5,"""DL""","""DL_CODESHARE""",19790,"""DL""",3298,"""9E""",20363,"""9E""","""N800AY""",3298,10146,1014602,30146,"""Albany, GA""","""GA""",13,"""Georgia""",34,10397,1039707,30397,"""Atlanta, GA""","""GA""",13,"""Georgia""",34,0,-1,"""1200-1259""",17,1207,1242,11,1304,-11,0,-1,"""1300-1359""",1,0
"""2018-01-27""","""Endeavor Air Inc.""","""ABY""","""ATL""",false,false,1400,1355,0,-5,1459,0,36,60,64,145,2018,1,1,27,6,"""DL""","""DL_CODESHARE""",19790,"""DL""",3298,"""9E""",20363,"""9E""","""N8839E""",3298,10146,1014602,30146,"""Albany, GA""","""GA""",13,"""Georgia""",34,10397,1039707,30397,"""Atlanta, GA""","""GA""",13,"""Georgia""",34,0,-1,"""1400-1459""",17,1412,1448,11,1500,-1,0,-1,"""1500-1559""",1,0


time: 102 ms (started: 2024-09-17 04:47:43 +00:00)


In [37]:
flight_df.select(pl.col("AirTime").sum()).collect()

AirTime
f64
614784103


time: 17.3 s (started: 2024-09-17 04:47:48 +00:00)


In [38]:
flight_df.select(pl.col("AirTime").sum()).collect(engine="gpu")

AirTime
f64
614784103


time: 7.62 s (started: 2024-09-17 04:48:06 +00:00)


In [39]:
gpu_engine = pl.GPUEngine(
    device=0,
    raise_on_fail=True,
)

time: 558 µs (started: 2024-09-17 04:48:13 +00:00)


# Find the top flight routes (based on origin, destination, month, and airline) that have the **longest average flight distances** & display only the top results.

In [40]:
res_cpu = (
    flight_df
    .group_by("Origin",	"Dest", "Month", "Airline",)
    .agg(pl.col("Distance").mean())
    .sort(by="Distance", descending=True)
    .head()
    .collect()
)

res_cpu

Origin,Dest,Month,Airline,Distance
str,str,i64,str,f64
"""JFK""","""HNL""",1,"""Hawaiian Airlines Inc.""",4983
"""HNL""","""JFK""",11,"""Hawaiian Airlines Inc.""",4983
"""HNL""","""JFK""",12,"""Delta Air Lines Inc.""",4983
"""HNL""","""JFK""",8,"""Hawaiian Airlines Inc.""",4983
"""JFK""","""HNL""",6,"""Hawaiian Airlines Inc.""",4983


time: 7.3 s (started: 2024-09-17 04:48:13 +00:00)


In [46]:
res_gpu = (
    flight_df
    .group_by("Origin",	"Dest", "Month", "Airline",)
    .agg(pl.col("Distance").median())
    .sort(by="Distance", descending=True)
    .head()
    .collect(engine=gpu_engine)
)
res_gpu

Origin,Dest,Month,Airline,Distance
str,str,i64,str,f64
"""HNL""","""JFK""",1,"""Delta Air Lines Inc.""",4983
"""HNL""","""JFK""",1,"""Hawaiian Airlines Inc.""",4983
"""HNL""","""JFK""",2,"""Hawaiian Airlines Inc.""",4983
"""HNL""","""JFK""",3,"""Hawaiian Airlines Inc.""",4983
"""HNL""","""JFK""",4,"""Hawaiian Airlines Inc.""",4983


time: 1.74 s (started: 2024-09-17 04:48:46 +00:00)


time: 2.8 s (started: 2024-09-17 04:48:21 +00:00)


# Find the **top 10 flight routes (by airline, origin, destination, and month)** that have the highest number of cancellations.

In [42]:
query = """
SELECT Airline, Origin,	Dest, Month, SUM(Cancelled) as total_cancelled
FROM flight_df
GROUP BY Airline, Origin,	Dest, Month
ORDER BY total_cancelled desc
LIMIT 50
"""

time: 450 µs (started: 2024-09-17 04:48:23 +00:00)


In [43]:
pl.sql(query).collect()

Airline,Origin,Dest,Month,total_cancelled
str,str,str,i64,u32
"""SkyWest Airlines Inc.""","""ASE""","""DEN""",7,62
"""SkyWest Airlines Inc.""","""DEN""","""ASE""",7,53
"""Comair Inc.""","""CLT""","""OAJ""",9,48
"""Comair Inc.""","""OAJ""","""CLT""",9,47
"""United Air Lines Inc.""","""BOS""","""EWR""",3,43
…,…,…,…,…
"""Southwest Airlines Co.""","""HOU""","""DAL""",8,29
"""Horizon Air""","""SEA""","""MFR""",10,29
"""American Airlines Inc.""","""ILM""","""CLT""",9,29


time: 7.17 s (started: 2024-09-17 04:48:23 +00:00)


In [44]:
pl.sql(query).collect(engine=gpu_engine)

Airline,Origin,Dest,Month,total_cancelled
str,str,str,i64,i64
"""SkyWest Airlines Inc.""","""ASE""","""DEN""",7,62
"""SkyWest Airlines Inc.""","""DEN""","""ASE""",7,53
"""Comair Inc.""","""CLT""","""OAJ""",9,48
"""Comair Inc.""","""OAJ""","""CLT""",9,47
"""United Air Lines Inc.""","""BOS""","""EWR""",3,43
…,…,…,…,…
"""Southwest Airlines Co.""","""HOU""","""DAL""",8,29
"""Delta Air Lines Inc.""","""CHS""","""ATL""",9,29
"""Comair Inc.""","""FAY""","""CLT""",9,29


time: 1.68 s (started: 2024-09-17 04:48:31 +00:00)
